In [2]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
import pandas as pd
import os
import numpy as np
import matplotlib.pyplot as plt
import sys

In [4]:
sys.path.insert(0, os.path.abspath('..'))
from UKMovementSensing import hsmm
from milleniumcohort import create_config

In [6]:
config = create_config('../config.yml')
config.create_data_paths()

## Load model

In [8]:
import pickle
with open(config.model_file, 'r') as f:
    model = pickle.load(file=f)

In [9]:
for o in model.obs_distns:
    print(o.params)

{'mu': array([ 0.01504721]), 'sigma': array([[ 0.00096502]])}
{'mu': array([ 0.0058569]), 'sigma': array([[ 0.0004863]])}
{'mu': array([ 0.07824656]), 'sigma': array([[ 0.02387891]])}
{'mu': array([ 0.04052145]), 'sigma': array([[ 0.00500122]])}


In [11]:
means = pd.DataFrame([o.params['mu'] for o in model.obs_distns], columns=config.hsmmconfig.column_names)
means.transpose().style.background_gradient(cmap='coolwarm',axis=1)

In [13]:
sigmas = pd.DataFrame([o.params['sigma'][0] for o in model.obs_distns], columns=config.hsmmconfig.column_names)
sigmas.transpose().style.background_gradient(cmap='coolwarm',axis=1)

In [14]:
durations = pd.DataFrame([o.params['lmbda'] for o in model.dur_distns], columns=['duration'])
durations['duration_sec'] = ["{} sec".format(int(round(x*5))) for x in durations['duration']]
durations.transpose()

,0,1,2,3
duration,374,336.151,348.541,343.16
duration_sec,1870 sec,1681 sec,1743 sec,1716 sec


In [15]:
# Transition probablilites
pd.DataFrame(model.trans_distn.trans_matrix).style.background_gradient(cmap='BuGn')

In [16]:
concat = pd.concat([means, sigmas], axis=1, keys=['mean', 'sigma'])
concat = concat.reorder_levels(order=[1,0], axis=1)
concat[('duration', 'lmbda')] = durations['duration_sec']

In [17]:
concat

acceleration            duration
          mean     sigma     lmbda
0     0.015047  0.000965  1870 sec
1     0.005857  0.000486  1681 sec
2     0.078247  0.023879  1743 sec
3     0.040521  0.005001  1716 sec

In [18]:
concat.transpose().to_clipboard()